In [3]:
import numpy as np
import torch 

In [8]:
obj = lambda x: x[0]**2 + (x[1] - 3)**2
g1 = lambda x: x[1]**2 - 2*x[0]
g2 = lambda x: (x[1]-1)**2 + 5*x[0]-15
df_obj  = lambda x : torch.tensor([[2 * x[0], 2 * (x[1] - 3)]])
df_g1 = lambda x : torch.tensor([[-2, 2 * x[1]]])
df_g2 = lambda x: torch.tensor([[5, 2 * (x[1] - 1)]])
x = torch.tensor([[1,1.]]).T
Lambda = torch.zeros((x.shape[0], 1))
weights = torch.zeros((x.shape[0], 1))+2
s = torch.tensor([[1,1.]]).T

In [9]:
def meritFun(Lambda, weights,k):
  if k > 0:
    weights = torch.max(abs(Lambda), 0.5 *(weights + abs(Lambda)))
  else:
    weights = abs(Lambda)
  return weights

In [22]:
def F_alpha(x,weights,alpha,s):
  G1=max(0, g1(x + alpha*s)) # constrain 1
  G2=max(0, g2(x + alpha*s)) # constrain 2
  return obj(x + alpha*s) +  weights[0,:] * G1 + weights[1,:]* G2

In [25]:
def lineSearch(x,Lambda,weights,s,k):
    t = 0.3
    alpha = 1
  
    if g1(x) <= 0:
       dg1_da = 0
    else:
      dg1_da= np.matmul(df_g1, s)

    if g2(x) <= 0:
       dg2_da = 0
    else:
      dg2_da= np.matmul(df_g2, s)

    dF_da = np.matmul(df_obj, s) + (weights[0, :] * dg1_da + weights[1, :] * dg2_da)

    phi = lambda x, weights, alpha, t, dF_da: F_alpha(x, weights, 0, 0) + alpha * t * dF_da

    while phi(x, weights, alpha, t, dF_da) < F_alpha(x, weights, alpha, s):
        alpha = 0.5 * alpha
    return alpha, weights

In [26]:
def BFGS(x,W, s, Lambda, alpha):
  lx_k = df_obj(x) + torch.matmul(Lambda.T, torch.tensor([g1(x),g2(x)]))
  lx_k_1 = df_obj(x + alpha*s) + torch.matmul(Lambda.T, torch.tensor([g1(x + alpha*s),g2(x + alpha*s )]))

  delta_l = lx_k_1 -lx_k
  # print(delta_l)
  Q = torch.matmul(torch.matmul((alpha*s).T, W), (alpha*s))
  if torch.matmul((alpha*s).T, delta_l.T) >= 0.2 * torch.matmul(torch.matmul((alpha*s).T, W), (alpha*s)):
        theta = 1
  else:
        theta = 0.8 * Q / (Q - torch.matmul((alpha*s).T, delta_l.T))
  y = theta * delta_l.T + (1 - theta) * torch.matmul(W, (alpha*s))
  W = W + torch.matmul(y, y.T) / torch.matmul(y.T, s) - torch.matmul(torch.matmul(W, s), torch.matmul(s.T, W)) / torch.matmul(torch.matmul(s.T, W), s)
  return W
W = torch.eye(x.shape[0])
k = 0
alpha = 1
